In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import environment as env

In [2]:
try:
    conn_string = "dbname=%s user=%s host=%s password=%s port=%s" % (env.DB_NAME, env.USERNAME, env.HOST, env.PASSWORD, env.PORT)
    conn = psycopg2.connect(conn_string)
    print("Conexión exitosa!")
except:
    print("No se pudo realizar la conexión a la BD.")

Conexión exitosa!


In [3]:
# Calculate the WAU - weekly unique active users (where an active = a user that plays an audiobook) for the dates provided in the data.
query = """
    SELECT
        DATE_PART('isoyear', created_at) AS Year,
        DATE_PART('week', created_at) AS Week,
        COUNT(user_id) AS WAU
    FROM 
        audiobook_plays
    GROUP BY 
        DATE_PART('isoyear', created_at), 
        DATE_PART('week', created_at), 
        user_id
    HAVING 
        COUNT(audiobook_id) > 1
    ORDER BY 
        DATE_PART('isoyear', created_at) DESC, DATE_PART('week', created_at) DESC;
"""
wau = pd.read_sql(query, conn)
print(wau)

        year  week  wau
0     2019.0  15.0   15
1     2019.0  15.0    6
2     2019.0  15.0   61
3     2019.0  15.0   17
4     2019.0  15.0    2
...      ...   ...  ...
5254  2018.0  49.0    6
5255  2018.0  49.0    5
5256  2018.0  49.0    4
5257  2018.0  49.0   18
5258  2018.0  49.0   79

[5259 rows x 3 columns]


In [4]:
# What is the average time listened (in hours) per user for the last 30 days for the data provided?
query = """
    WITH max_date AS ( 
        SELECT 
            MAX(created_at) as max_date 
        FROM 
            audiobook_plays
        LIMIT 1
    )
    SELECT
        a.user_id,
        ROUND(AVG(a.seconds) / 3600, 5) as avg_hours
    FROM audiobook_plays a, max_date b
    WHERE
        a.created_at BETWEEN b.max_date - INTERVAL '30 DAY' AND b.max_date
    GROUP BY
        a.user_id
    ORDER BY
        ROUND(AVG(a.seconds) / 3600, 2) DESC
    LIMIT 10
"""
avg_listen = pd.read_sql(query, conn)
print(avg_listen)

   user_id  avg_hours
0   983885   16.27611
1   991429   11.79961
2   971029   11.39840
3   966613    7.66967
4   974151    7.64907
5   986939    6.81231
6   904922    6.12468
7   951726    5.77219
8   979560    5.69308
9   971059    5.57944


In [5]:
# What is the book category that has been showing more popularity?
# Why and how did you picked it?
query = """
    WITH categories AS (
        SELECT 
	        id, 
	        UNNEST(book_category_codes) AS category
        FROM 
            audiobook
    ),
    category_names AS (
        SELECT
	        A.*,
	        B.name
        FROM
	        categories A
        LEFT JOIN
	        book_categories B
        ON
	        A.category = B.book_cateogory_code 
    ),
    rankings AS (
    SELECT
        DATE_PART('isoyear', created_at) AS Year,
        DATE_PART('week', created_at) AS Week,
        B.name AS category,
        COUNT(B.name) AS count
    FROM 
        audiobook_plays A
    LEFT JOIN 
    	category_names B
    ON
    	A.audiobook_id = B.id 
    WHERE
    	B.name is not null
    GROUP BY 
        DATE_PART('isoyear', created_at), 
        DATE_PART('week', created_at),
        B.name
    ORDER BY 
        B.name desc,
        DATE_PART('week', created_at) DESC,
    	DATE_PART('isoyear', created_at)	
    ),
    previous_count AS (
    SELECT 
    *,
    LAG(count, 1)  OVER (ORDER BY category, week, year) AS previous_count
    FROM rankings 
    ),
    grow_index AS (
    SELECT *, 
        CASE COALESCE(previous_count, 0)
            WHEN 0 THEN 0
            WHEN count THEN 0
            ELSE (count * 100) / previous_count
        END AS grow_percent
     FROM previous_count
    )
    SELECT * FROM grow_index ORDER BY year desc, week desc, grow_percent desc LIMIT 1;
"""
popularity = pd.read_sql(query, conn)
popularity.head()

,year,week,category,count,previous_count,grow_percent
0,2019.0,15.0,Young Adult Fiction / Fantasy / Wizards & Witches,42,1,4200


# Find two quality issues that affect the integrity of the database
1. La tabla **book_categories** no está relacionada con la tabla **audiobook** esto puede provocar problemas de consistencia en los datos al momento de realizar actualizaciones / eliminaciones en las categorías.
2. La tabla **audiobook_plays** que es la que tiene mas registros y representa una mayor transaccionalidad no tiene creados campos de partición, esto al momento de tener un aumento considerable de registros las operaciones serán cada vez mas pesadas dado que se aplican los *queries* en un solo fichero; mi recomendación personal es manejar particiones y utilizar estos campos para manejar consultas mas eficientes.
